In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from tabulate import tabulate
from finance_byu.rolling import roll_sum
from finance_byu.summarize import summary
from finance_byu.regtables import Regtable

# Initiations:

In [2]:
pd.options.display.max_rows = 100

# Read in the monthly CRSP data, clean, and set up features
stk = pd.read_feather('C:/Users/jarem/Downloads/crsp_monthly.ftr')
stk['mdt'] = stk['caldt'].values.astype('datetime64[M]')

stk['prc'] = stk['prc'].abs()
stk['me'] = stk.eval("prc*shr/1000.0").where(stk.eval("prc*shr > 1e-6")) 

stk['prclag'] = stk.groupby('permno')['prc'].shift()
stk['melag'] = stk.groupby('permno')['me'].shift()
stk

,permno,caldt,cusip,ticker,shrcd,excd,siccd,prc,ret,vol,shr,cumfacshr,mdt,me,prclag,melag
0,10000,1986-01-31,68391610,OMFGA,10,3,3990,4.37500,NaN,1771.0,3680.0,1.0,1986-01-01,1.610000e+01,NaN,NaN
1,10000,1986-02-28,68391610,OMFGA,10,3,3990,3.25000,-0.257143,828.0,3680.0,1.0,1986-02-01,1.196000e+01,4.37500,1.610000e+01
2,10000,1986-03-31,68391610,OMFGA,10,3,3990,4.43750,0.365385,1078.0,3680.0,1.0,1986-03-01,1.633000e+01,3.25000,1.196000e+01
3,10000,1986-04-30,68391610,OMFGA,10,3,3990,4.00000,-0.098592,957.0,3793.0,1.0,1986-04-01,1.517200e+01,4.43750,1.633000e+01
4,10000,1986-05-30,68391610,OMFGA,10,3,3990,3.10938,-0.222656,1074.0,3793.0,1.0,1986-05-01,1.179388e+01,4.00000,1.517200e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5004038,93436,2024-08-30,88160R10,TSLA,11,3,9999,214.11000,-0.077391,16108365.0,3194640.0,1.0,2024-08-01,6.840044e+05,232.07001,7.413801e+05
5004039,93436,2024-09-30,88160R10,TSLA,11,3,9999,261.63000,0.221942,16042065.0,3207000.0,1.0,2024-09-01,8.390474e+05,214.11000,6.840044e+05
5004040,93436,2024-10-31,88160R10,TSLA,11,3,9999,249.85001,-0.045025,19014312.0,3210060.0,1.0,2024-10-01,8.020335e+05,261.63000,8.390474e+05
5004041,93436,2024-11-29,88160R10,TSLA,11,3,9999,345.16000,0.381469,20821313.0,3210060.0,1.0,2024-11-01,1.107984e+06,249.85001,8.020335e+05


In [3]:
# Read in the dividends data, clean, and set up features
div = pd.read_csv('C:/Users/jarem/Downloads/dividends_cash.csv', parse_dates=['paydt'])
div['mdt'] = div['paydt'].values.astype('datetime64[M]')
div = div[['permno','mdt','divamt']]
div

,permno,mdt,divamt
0,10001,1986-03-01,0.095
1,10001,1986-06-01,0.105
2,10001,1986-09-01,0.105
3,10001,1986-12-01,0.105
4,10001,1987-03-01,0.105
...,...,...,...
985382,93429,2023-12-01,0.550
985383,93429,2024-03-01,0.550
985384,93429,2024-06-01,0.550
985385,93429,2024-09-01,0.630


In [4]:
# Merge the data and lag 1 month to avoid counting the current month
stk = stk.merge(div, on=['permno','mdt'], how='left')
stk['div'] = stk['divamt'].notnull().astype(int)
stk

,permno,caldt,cusip,ticker,shrcd,excd,siccd,prc,ret,vol,shr,cumfacshr,mdt,me,prclag,melag,divamt,div
0,10000,1986-01-31,68391610,OMFGA,10,3,3990,4.37500,NaN,1771.0,3680.0,1.0,1986-01-01,1.610000e+01,NaN,NaN,NaN,0
1,10000,1986-02-28,68391610,OMFGA,10,3,3990,3.25000,-0.257143,828.0,3680.0,1.0,1986-02-01,1.196000e+01,4.37500,1.610000e+01,NaN,0
2,10000,1986-03-31,68391610,OMFGA,10,3,3990,4.43750,0.365385,1078.0,3680.0,1.0,1986-03-01,1.633000e+01,3.25000,1.196000e+01,NaN,0
3,10000,1986-04-30,68391610,OMFGA,10,3,3990,4.00000,-0.098592,957.0,3793.0,1.0,1986-04-01,1.517200e+01,4.43750,1.633000e+01,NaN,0
4,10000,1986-05-30,68391610,OMFGA,10,3,3990,3.10938,-0.222656,1074.0,3793.0,1.0,1986-05-01,1.179388e+01,4.00000,1.517200e+01,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5041301,93436,2024-08-30,88160R10,TSLA,11,3,9999,214.11000,-0.077391,16108365.0,3194640.0,1.0,2024-08-01,6.840044e+05,232.07001,7.413801e+05,NaN,0
5041302,93436,2024-09-30,88160R10,TSLA,11,3,9999,261.63000,0.221942,16042065.0,3207000.0,1.0,2024-09-01,8.390474e+05,214.11000,6.840044e+05,NaN,0
5041303,93436,2024-10-31,88160R10,TSLA,11,3,9999,249.85001,-0.045025,19014312.0,3210060.0,1.0,2024-10-01,8.020335e+05,261.63000,8.390474e+05,NaN,0
5041304,93436,2024-11-29,88160R10,TSLA,11,3,9999,345.16000,0.381469,20821313.0,3210060.0,1.0,2024-11-01,1.107984e+06,249.85001,8.020335e+05,NaN,0


In [5]:
# Calculate dividend indicators
# d12: dividends in past 12 months (lagged)
stk['d12'] = (stk.groupby('permno')['div'].rolling(12).sum().reset_index(drop=True))
stk['d12'] = stk.groupby('permno')['d12'].shift(1)

# div60to13: dividends in months -60 to -13 (5 years prior to last year)
stk['div60to13'] = (stk.groupby('permno')['div'].rolling(48,12).sum().reset_index(drop=True))
stk['div60to13'] = stk.groupby('permno')['div60to13'].shift(13)

# div24to13: dividends in months -24 to -13 (for omission criteria)
stk['div24to13'] = (stk.groupby('permno')['div'].rolling(24).sum().reset_index(drop=True))
stk['div24to13'] = stk.groupby('permno')['div24to13'].shift(13)

# Create portfolio indicators
stk['omit'] = stk.eval("div24to13 == 4 and d12 < 4").astype(int)
stk['init'] = stk.eval("d12 > 0 and div60to13 == 0").astype(int)

In [6]:
# Assign portfolios
stk['port'] = 0
stk['port'] = stk['port'].where(stk['omit'] == 0, 1)  # Omissions get port=1
stk['port'] = stk['port'].where(stk['init'] == 0, 2)  # Initiations get port=2 (overriding omissions)

# Filter the data
stk = (stk.query("10 <= shrcd <= 11 and melag == melag and prclag > 4.999 and ret == ret")
       .reset_index(drop=True))

In [7]:
# Summary statistics
print("\nSummary Statistics")
print("Equal-Weight Portfolios\n")

# Calculate mean returns, standard deviations, and t-stats for each portfolio
portfolio_stats = []
for port in [0, 1, 2]:
    port_data = stk[stk['port'] == port]
    mean_ret = port_data['ret'].mean() * 100
    std_dev = port_data['ret'].std() * 100
    t_stat = mean_ret / (std_dev / np.sqrt(len(port_data)))
    
    portfolio_stats.append({
        'Portfolio': f'p{port}',
        'Mean': f"{mean_ret:.3f}",
        'Std Dev': f"{std_dev:.3f}", 
        't-stat': f"{t_stat:.3f}"
    })


Summary Statistics
Sample Period: 1928:01-2024:09
Equal-Weight Portfolios



In [8]:
# Print results in a table format
print(tabulate(portfolio_stats, headers="keys", tablefmt="simple"))

Portfolio      Mean    Std Dev    t-stat
-----------  ------  ---------  --------
p0            0.974     13.23    123.158
p1            1.259     12.033    22.54
p2            1.536     12.941    24.865
